In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from numpy import nan

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('max_seq_item', None)

In [7]:
# DATA SOURCE: INFORMATION AND CONDITION OF SCHOOLS (ICOS), Pre-Disaster Mitigation module
# Data received from WA Office of Superintendent of Public Instruction (OSPI), 8/14/24, via public records request.
# https://www.k12.wa.us/policy-funding/school-buildings-facilities/information-and-condition-schools-icos
df_icos_original=pd.read_csv("data/df_icos_clean_8.14.24.csv")

In [8]:
# Copy the data and work with the copy.
df_icos = df_icos_original

In [9]:
df_icos['Building_YearBuilt'] = df_icos['Building_YearBuilt'].astype('Int64')
df_icos['BuildingArea_YearBuilt'] = df_icos['BuildingArea_YearBuilt'].astype('Int64')

In [10]:
df_icos_small = df_icos[['District', 'Site', 'Building', 'Building_YearBuilt', \
                         'BuildingArea_YearBuilt', 'BuildingArea_CodeYear', \
                         'BuildingArea_HasRetrofit']].drop_duplicates()

In [11]:
df_icos_small.shape

(6165, 7)

In [12]:
# Were seismic upgrades that were identified in the SSSP phase 1 incorporated into ICOS?

In [13]:
# DATA SOURCE 2: SCHOOL SEISMIC SAFETY PROJECT phase 1 engineering reports
# https://fortress.wa.gov/dnr/geologydata/school_seismic_safety/SSSP_2019_Engineering_Vol1_Engineering_Report.pdf

In [111]:
df_sssp_phase1=pd.read_csv("data/sssp_2019_engineering_reports.csv")

In [112]:
# Create a unique ID column because there are duplicate school names in different districts.
df_sssp_phase1['Unique_Identifier'] = df_sssp_phase1['District Name'].\
       astype(str) +"_"+ df_sssp_phase1['Facility Name'].astype(str)

In [113]:
# Move the unique ID to the first column
df_sssp_phase1 = df_sssp_phase1[ ['Unique_Identifier'] + [ col for col in df_sssp_phase1.columns \
                                                          if col != 'Unique_Identifier' ] ]

In [114]:
df_sssp_phase1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 28 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Unique_Identifier                               222 non-null    object 
 1   District Name                                   222 non-null    object 
 2   ICOS Facility ID                                222 non-null    int64  
 3   Facility Name                                   222 non-null    object 
 4   ICOS Building ID                                218 non-null    float64
 5   Building Name                                   222 non-null    object 
 6   Enrollment                                      222 non-null    int64  
 7   Latitude                                        222 non-null    float64
 8   Longitude                                       222 non-null    float64
 9   FEMA Construction Type                     

In [115]:
df_sssp_phase1['Year Built'] = df_sssp_phase1['Year Built'].astype('Int64')

In [116]:
df_sssp_phase1_upgrade = df_sssp_phase1[['District Name', 'Facility Name', 'Building Name', \
             'Year Built', 'Last Major Renovation', 'Did Renovation Include Seismic Upgrades? (Y/N)']]

In [117]:
df_sssp_phase1_upgrade['Did Renovation Include Seismic Upgrades? (Y/N)'].value_counts()

Did Renovation Include Seismic Upgrades? (Y/N)
No         79
Yes        25
Unknown    20
Name: count, dtype: int64

In [118]:
mask = (df_sssp_phase1_upgrade['Did Renovation Include Seismic Upgrades? (Y/N)'] == 'Yes')
df_sssp_phase1_upgrade_2 = df_sssp_phase1_upgrade[mask]

In [119]:
df_sssp_phase1_upgrade_2['Did Renovation Include Seismic Upgrades? (Y/N)'].value_counts()
# Phase 1 evaluations in the SSSP identified 25 buildings with prior seismic upgrades

Did Renovation Include Seismic Upgrades? (Y/N)
Yes    25
Name: count, dtype: int64

In [120]:
df_sssp_phase1_upgrade_2 = df_sssp_phase1_upgrade_2.rename(columns={'District Name': 'District', \
                          'Facility Name': 'Site', 'Building Name': 'Building'})

In [121]:
# Edit name variations in the SSSP list to match ICOS for merging
df_sssp_phase1_upgrade_2['Site'] = df_sssp_phase1_upgrade_2['Site'].replace\
          (["Clallam Bay High and Elementary School"], "Clallam Bay K-12 School")

In [122]:
df_sssp_phase1_upgrade_2.drop([212], axis=0, inplace=True)
#drop one row that has a blank for last renovation

In [123]:
df_sssp_phase1_upgrade_2.shape

(24, 6)

In [124]:
# create an ID for each building
df_sssp_phase1_upgrade_2['building_id'] =  df_sssp_phase1_upgrade_2.groupby(["District", "Site", \
                                                                             "Building"]).ngroup()

In [125]:
# Move the building ID to the first column
df_sssp_phase1_upgrade_2 = df_sssp_phase1_upgrade_2[ ['building_id'] + [ col for col in \
                                   df_sssp_phase1_upgrade_2.columns if col != 'building_id' ] ]

In [126]:
df_sssp_phase1_upgrade_2['building_id'].nunique()

24

In [127]:
# Merge these rows with ICOS rows
df_sssp_phase1_merge = df_sssp_phase1_upgrade_2.merge(df_icos_small, how = 'inner', on = \
                                                      ['District', 'Site', 'Building'])

In [128]:
df_sssp_phase1_merge.rename(columns={'building_id': 'building_id', 'Year Built': 'Year Built (SSSP)', 'Last Major Renovation': \
                                  'Last Major Renovation (SSSP)', \
                                  'Did Renovation Include Seismic Upgrades? (Y/N)': \
                                  'Did Renovation Include Seismic Upgrades? (Y/N) (SSSP)', \
                                  'Building_YearBuilt': 'Building_YearBuilt (ICOS)', \
                                  'BuildingArea_YearBuilt': 'BuildingArea_YearBuilt (ICOS)', \
                                  'BuildingArea_CodeYear': 'BuildingArea_CodeYear (ICOS)', \
                                  'BuildingArea_HasRetrofit': 'BuildingArea_HasRetrofit (ICOS)'}, inplace=True)

In [129]:
df_sssp_phase1_merge.shape # the 24 buildings in the SSSP matched to 33 rows in ICOS because some buildings
# have more than one "building area" row.

(33, 11)

In [130]:
df_sssp_phase1_merge['Last Major Renovation (SSSP)'] = df_sssp_phase1_merge['Last Major Renovation (SSSP)']\
              .astype('Int64')

In [131]:
df_sssp_phase1_merge['Did Renovation Include Seismic Upgrades? (Y/N) (SSSP)'].value_counts()
# All of these buildings have a seismic upgrade in the SSSP data

Did Renovation Include Seismic Upgrades? (Y/N) (SSSP)
Yes    33
Name: count, dtype: int64

In [132]:
df_sssp_phase1_merge['BuildingArea_HasRetrofit (ICOS)'].value_counts()
# But none of the building areas for the 24 buildings shows a retrofit in the ICOS data

BuildingArea_HasRetrofit (ICOS)
No    33
Name: count, dtype: int64

In [140]:
df_sssp_phase1_merge[['building_id', 'District', 'Site', 'Building', \
                      'Did Renovation Include Seismic Upgrades? (Y/N) (SSSP)', \
                      'BuildingArea_HasRetrofit (ICOS)']].drop_duplicates()

,building_id,District,Site,Building,Did Renovation Include Seismic Upgrades? (Y/N) (SSSP),BuildingArea_HasRetrofit (ICOS)
0,0,Cape Flattery,Clallam Bay K-12 School,Elementary Building,Yes,No
1,1,Creston,Creston Junior Senior High School,Creston K-12 School Building,Yes,No
3,2,Darrington,Darrington Senior High School,Darrington High School,Yes,No
5,3,Evaline,Evaline Elementary School,Main Building,Yes,No
7,4,Fife,Fife High School,Building IV 400 Library,Yes,No
8,5,Fife,Fife High School,Building IX 900 Science,Yes,No
9,6,Fife,Fife High School,Building V 500 Main,Yes,No
10,7,Fife,Fife High School,Building VI 600 Gyms,Yes,No
11,8,Fife,Fife High School,Building VII 700 Cafeteria,Yes,No
12,9,Fife,Fife High School,Building VIII 800 Shop,Yes,No


In [141]:
# Since the seismic upgrades were part of renovations, does ICOS show newer building areas for these buildings
# that match the renovation year to reflect the renovation that way?
df_sssp_phase1_merge['year_difference'] = df_sssp_phase1_merge['Last Major Renovation (SSSP)'] - \
             (df_sssp_phase1_merge['BuildingArea_YearBuilt (ICOS)'])

In [ ]:
# Does the renovation year in ICOS come around the same time as the renovation listed in the SSSP phase 1,
# such as no more than 2-3 years apart?

In [142]:
df_sssp_phase1_merge.loc[df_sssp_phase1_merge['year_difference'] > 2, 'upgrade_recorded'] = "no" 

In [143]:
df_sssp_phase1_merge.loc[df_sssp_phase1_merge['year_difference'] < 3, 'upgrade_recorded'] = "yes" 

In [144]:
df_sssp_phase1_merge = df_sssp_phase1_merge.reindex(['building_id', 'District', 'Site', 'Building', \
                       'upgrade_recorded', 'Year Built (SSSP)', 'Last Major Renovation (SSSP)', \
                       'Building_YearBuilt (ICOS)', 'BuildingArea_YearBuilt (ICOS)', \
                       'BuildingArea_CodeYear (ICOS)', 'BuildingArea_HasRetrofit (ICOS)', \
                       'year_difference'], axis=1)

In [145]:
# Remove any buildings with a recorded upgrade for any row
mask = (df_sssp_phase1_merge['upgrade_recorded'] == 'yes')
df_sssp_phase1_upgardes_recorded = df_sssp_phase1_merge[mask]

In [146]:
df_sssp_phase1_upgrades_recorded_ids = df_sssp_phase1_upgrades_recorded['building_id']

In [147]:
df_sssp_phase1_upgrades_recorded_ids.shape

(8,)

In [148]:
df_sssp_phase1_upgrade_missing = df_sssp_phase1_merge[~df_sssp_phase1_merge['building_id'].isin\
                                                                 (df_sssp_phase1_upgrades_recorded_ids)]

In [149]:
df_sssp_phase1_upgrade_missing['building_id'].nunique()

15

In [150]:
df_sssp_phase1_upgrade_missing
# These 15 buildings do not have a newer building area recorded around the same time as the renovation listed
# by the SSSP. The code years also do not show a renovation bringing the buildings up to a more recent code.

,building_id,District,Site,Building,upgrade_recorded,Year Built (SSSP),Last Major Renovation (SSSP),Building_YearBuilt (ICOS),BuildingArea_YearBuilt (ICOS),BuildingArea_CodeYear (ICOS),BuildingArea_HasRetrofit (ICOS),year_difference
0,0,Cape Flattery,Clallam Bay K-12 School,Elementary Building,no,1962,2006,1962,1962,<1973,No,44
7,4,Fife,Fife High School,Building IV 400 Library,no,1950,1975,1950,1950,<1973,No,25
8,5,Fife,Fife High School,Building IX 900 Science,no,1970,1992,1970,1970,<1973,No,22
9,6,Fife,Fife High School,Building V 500 Main,no,1950,1992,1950,1950,<1973,No,42
10,7,Fife,Fife High School,Building VI 600 Gyms,no,1956,1992,1956,1956,<1973,No,36
11,8,Fife,Fife High School,Building VII 700 Cafeteria,no,1963,1992,1963,1963,<1973,No,29
12,9,Fife,Fife High School,Building VIII 800 Shop,no,1963,1992,1963,1963,<1973,No,29
13,10,La Conner,La Conner High School,High School Auditorium,no,1921,2003,1921,1921,<1973,No,82
14,11,Ocean Beach,Ilwaco (Hilltop) Middle School,Auditorium,no,1936,2007,1936,1936,<1973,No,71
18,13,Pasco,Edwin Markham Elementary School,Main Building,no,1962,1984,1962,1962,<1973,No,22


In [46]:
# Out of 24 buildings in the SSSP phase 1 that are marked as having a prior seismic upgrade with a renovation year
# included, 0 buildings have a retrofit in ICOS, and 15 buildings have no recent build year or code upgrade to 
# reflect a renovation around the years notes in the SSSP.

In [47]:
# Were seismic upgrades that were identified in the SSSP phase 2 incorporated into ICOS?

In [249]:
# DATA SOURCE 3: SCHOOL SEISMIC SAFETY PROJECT phase 2 engineering reports
# https://fortress.wa.gov/dnr/geologydata/school_seismic_safety/phase2/SSSP_2021_Engineering_Vol1_Seismic_Assessment_Report.pdf
df_sssp_phase2=pd.read_csv("data/sssp_2021_engineering_reports.csv")

In [250]:
# Create a unique ID column because there are duplicate school names in different districts.
df_sssp_phase2['Unique_Identifier'] = df_sssp_phase2['District Name'].\
       astype(str) +"_"+ df_sssp_phase2['Site Name'].astype(str)

In [251]:
# Move the unique ID to the first column
df_sssp_phase2 = df_sssp_phase2[ ['Unique_Identifier'] + [ col for col in df_sssp_phase2.columns \
                                                          if col != 'Unique_Identifier' ] ]

In [246]:
df_sssp_phase2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338 entries, 0 to 337
Data columns (total 21 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Unique_Identifier                              338 non-null    object 
 1   District Name                                  338 non-null    object 
 2   ICOS Site ID                                   338 non-null    int64  
 3   Site Name                                      338 non-null    object 
 4   ICOS Bldg ID.No.                               338 non-null    int64  
 5   Building Name                                  338 non-null    object 
 6   Latitude                                       338 non-null    float64
 7   Longitude                                      338 non-null    float64
 8   FEMA Const. Type                               338 non-null    object 
 9   Year Built                                     338 non

In [252]:
df_sssp_phase2 = df_sssp_phase2.rename(columns={'District Name': 'District', \
                          'Site Name': 'Site', 'Building Name': 'Building'})

In [248]:
df_sssp_phase2

,Unique_Identifier,District,ICOS Site ID,Site,ICOS Bldg ID.No.,Building,Latitude,Longitude,FEMA Const. Type,Year Built,Last Renovation,Gross Area (SF),No. of Floors,"Structural Drawings Avail? (Yes, No, Partial)",Has Had Structural Upgrade?,Year of Structural Upgrade,Tsunami Risk,Vs30 Measured Site Class,Vs30 (m/s),"Sds, BSE-1N (g)",ASCE 41 Tier 1 Assessed By
0,Aberdeen_A.J. West Elementary School,Aberdeen,21443,A.J. West Elementary School,57384,1952 Building,46.972,-123.838,W2,1952,1952.0,"22,630",2,P,Yes,1994.0,YES,E,128.0,1.32,NaN
1,Aberdeen_A.J. West Elementary School,Aberdeen,21443,A.J. West Elementary School,57385,Annex Building,46.972,-123.838,W2,1966,1994.0,"16,400",1,P,Yes,1994.0,YES,E,128.0,1.32,WSP
2,Aberdeen_Central Park Elementary School,Aberdeen,21445,Central Park Elementary School,57391,Annex Building,46.968,-123.698,RM1,1966,1995.0,"5,895",1,P,No,NaN,NO,D,339.0,0.99,NaN
3,Aberdeen_Central Park Elementary School,Aberdeen,21445,Central Park Elementary School,57392,Main Building,46.968,-123.698,W2,1956,1995.0,"21,446",1,P,No,NaN,NO,D,339.0,0.99,WSP
4,Aberdeen_Hopkins Building (Harbor\rHigh School),Aberdeen,21446,Hopkins Building (Harbor\rHigh School),57394,Hopkins Building,46.972,-123.832,C2a,1956,NaN,"53,604",1,Y,No,NaN,YES,E,140.0,1.32,NaN
5,Aberdeen_J. M. Weatherwax High School,Aberdeen,21441,J. M. Weatherwax High School,57378,1964 Gymnasium Building,46.980,-123.818,RM1,1964,NaN,"27,409",3,N,No,NaN,YES,E,109.0,1.32,WSP
6,Aberdeen_J. M. Weatherwax High School,Aberdeen,21441,J. M. Weatherwax High School,57378,Main Building,46.980,-123.818,S2a,1964,NaN,"173,011",3,N,No,NaN,YES,E,109.0,1.32,NaN
7,Aberdeen_McDermoth Elementary School,Aberdeen,21448,McDermoth Elementary School,57397,Main Building,46.977,-123.823,W2,1926,1998.0,"61,867",3,P,Yes,1998.0,YES,D,234.0,1.01,WSP
8,Anacortes_Mount Erie Elementary School,Anacortes,20899,Mount Erie Elementary School,54084,Main Building,48.487,-122.619,RM1,1955,1991.0,"41,796",1,Y,No,NaN,NO,C,522.5,0.92,NaN
9,Bainbridge Island_Bainbridge High School,Bainbridge Island,21451,Bainbridge High School,57407,300 Building,47.637,-122.525,RM1,1981,NaN,"64,216",2,Y,Yes,1998.0,NO,D,295.0,0.98,WSP


In [254]:
# Edit name variations in the SSSP list to match ICOS for merging
df_sssp_phase2['Building'] = df_sssp_phase2['Building'].replace\
                          (["Main Building - Classrooms"], "Main Building")
df_sssp_phase2['Building'] = df_sssp_phase2['Building'].replace\
                         (["High School 100 Bldg A -\rNorth Wing"], "High School 100 A building")
df_sssp_phase2['Building'] = df_sssp_phase2['Building'].replace\
                       (["High School 100 Bldg A -\rSouth Wing"], "High School 100 A building")
df_sssp_phase2['Site'] = df_sssp_phase2['Site'].replace(["Southern Heights\rElementary School"], \
                                                          "Southern Heights Elementary School")
df_sssp_phase2['Building'] = df_sssp_phase2['Building'].replace\
                      (["Building 2 -\rClassrooms/Kitchen"], "Building 2 - Classrooms/Kitchen")
df_sssp_phase2['Site'] = df_sssp_phase2['Site'].replace(["Oak Harbor Middle School"], \
                                                          "Oak Harbor Intermediate School")
df_sssp_phase2['Building'] = df_sssp_phase2['Building'].replace\
                    (["Gymnasium"], "Gym/Gym Add/Boiler - AA Areas 1,2,3,4")
df_sssp_phase2['Building'] = df_sssp_phase2['Building'].replace\
                    (["Cafeteria/Gym"], "Cafeteria/Gym - AA Areas 24-28")
df_sssp_phase2['Site'] = df_sssp_phase2['Site'].replace\
               (["South Whidbey Grades 5 & 6 -\r(Formerly S. Whid. Primary)"], \
                "South Whidbey High School/Middle School Campus")

In [255]:
df_sssp_phase2['Year of Structural Upgrade'] = df_sssp_phase2['Year of Structural Upgrade'].\
        astype('Int64')

In [256]:
df_sssp_phase2['Last Renovation'] = df_sssp_phase2['Last Renovation'].\
        astype('Int64')

In [257]:
df_sssp_phase2_small = df_sssp_phase2[['District', 'Site', 'Building', \
             'Year Built', 'Last Renovation', 'Has Had Structural Upgrade?', 'Year of Structural Upgrade']]

In [258]:
df_sssp_phase2_small['Has Had Structural Upgrade?'].value_counts()

Has Had Structural Upgrade?
No     304
Yes     34
Name: count, dtype: int64

In [259]:
mask = (df_sssp_phase2_small['Has Had Structural Upgrade?'] == 'Yes')
df_sssp_phase2_upgrade = df_sssp_phase2_small[mask]

In [260]:
df_sssp_phase2_upgrade.shape

(34, 7)

In [261]:
# This list now consists of only buildings that are marked yes as having had structural upgrades,
# so that column can be deleted
df_sssp_phase2_upgrade = df_sssp_phase2_upgrade.drop(['Has Had Structural Upgrade?'], axis=1)

In [262]:
df_sssp_phase2_upgrade['building_id'] =  df_sssp_phase2_upgrade.groupby(["District", "Site", "Building"]).ngroup()

In [263]:
df_sssp_phase2_upgrade['building_id'].nunique() 

33

In [264]:
# Move the building ID to the first column
df_sssp_phase2_upgrade = df_sssp_phase2_upgrade[ ['building_id'] + [ col for col in \
                                   df_sssp_phase2_upgrade.columns if col != 'building_id' ] ]

In [265]:
# Merge these rows with ICOS rows
df_sssp_phase2_merge = df_sssp_phase2_upgrade.merge(df_icos_small, how = 'inner', on = \
                                                      ['District', 'Site', 'Building'])

In [266]:
df_sssp_phase2_merge.rename(columns={'building_id': 'building_id', 'Year Built': 'Year Built (SSSP)', \
                                  'Last Renovation': 'Last Renovation (SSSP)', 'Year of Structural Upgrade': \
                                  'Year of Structural Upgrade (SSSP)', 'Building_YearBuilt': \
                                  'Building_YearBuilt (ICOS)', 'BuildingArea_YearBuilt': \
                                  'BuildingArea_YearBuilt (ICOS)', 'BuildingArea_CodeYear': \
                                  'BuildingArea_CodeYear (ICOS)',  'BuildingArea_HasRetrofit': \
                                  'BuildingArea_HasRetrofit (ICOS)'}, inplace=True)

In [267]:
df_sssp_phase2_merge['BuildingArea_HasRetrofit (ICOS)'].value_counts()

BuildingArea_HasRetrofit (ICOS)
No            36
Structural     8
Name: count, dtype: int64

In [268]:
mask = (df_sssp_phase2_merge['BuildingArea_HasRetrofit (ICOS)'] == "Structural")
df_sssp_phase2_retrofit = df_sssp_phase2_merge[mask]
df_sssp_phase2_retrofit['building_id'].nunique()
# Only 5 out of 33 buildings listed as having a structural upgrade in the SSSP phase 2 have a retrofit in ICOS.

5

In [270]:
df_sssp_phase2_merge['building_id'].nunique() # Not all of the 33 buildings could be matched in ICOS due
# to different building names

26

In [271]:
df_sssp_phase2_merge.shape # the buildings in the SSSP matched to 44 rows in ICOS because some buildings
# have more than one "building area" row.

(44, 11)

In [272]:
df_sssp_phase2_merge['Year of Structural Upgrade (SSSP)'] = df_sssp_phase2_merge\
                    ['Year of Structural Upgrade (SSSP)'].astype('Int64')

In [273]:
df_sssp_phase2_merge['year_difference'] = df_sssp_phase2_merge['Year of Structural Upgrade (SSSP)'] - \
            (df_sssp_phase2_merge['BuildingArea_YearBuilt (ICOS)'])

In [ ]:
# Does the renovation year in ICOS come around the same time as the renovation listed in the SSSP phase 2,
# such as no more than 2-3 years apart?

In [72]:
df_sssp_phase2_merge.loc[df_sssp_phase2_merge['year_difference'] > 2, 'upgrade_recorded'] = "no" 

In [73]:
df_sssp_phase2_merge.loc[df_sssp_phase2_merge['year_difference'] < 3, 'upgrade_recorded'] = "yes" 

In [74]:
 df_sssp_phase2_merge = df_sssp_phase2_merge.reindex(['building_id', 'District', 'Site', 'Building', \
                        'upgrade_recorded', 'Year Built (SSSP)', 'Year of Structural Upgrade (SSSP)', \
                        'Building_YearBuilt (ICOS)', 'BuildingArea_YearBuilt (ICOS)', \
                        'BuildingArea_CodeYear (ICOS)', 'BuildingArea_HasRetrofit (ICOS)'], axis=1)

In [75]:
mask = (df_sssp_phase2_merge['BuildingArea_HasRetrofit (ICOS)'] == 'No')
df_sssp_phase2_merge_2 = df_sssp_phase2_merge[mask]
df_sssp_phase2_merge_2.shape

(36, 11)

In [76]:
df_sssp_phase2_merge_2['BuildingArea_HasRetrofit (ICOS)'].value_counts()

BuildingArea_HasRetrofit (ICOS)
No    36
Name: count, dtype: int64

In [77]:
df_sssp_phase2_merge_2['upgrade_recorded'].value_counts()

upgrade_recorded
no     28
yes     8
Name: count, dtype: int64

In [274]:
# Remove buildings that show a newer building area in the vicinity of the renovation recorded in the SSSP
mask = (df_sssp_phase2_merge_2['upgrade_recorded'] == 'yes')
df_sssp_phase2_upgrades_recorded = df_sssp_phase2_merge_2[mask]

In [275]:
df_sssp_phase2_upgrades_recorded.shape

(8, 11)

In [276]:
df_sssp_phase2_upgrades_recorded_ids = df_sssp_phase2_upgrades_recorded['building_id']

In [277]:
df_sssp_phase2_upgrades_recorded_ids.shape

(8,)

In [278]:
df_sssp_phase2_upgrade_missing = df_sssp_phase2_merge_2[~df_sssp_phase2_merge_2['building_id'].isin\
                                                                 (df_sssp_phase2_upgrades_recorded_ids)]

In [279]:
df_sssp_phase2_upgrade_missing['building_id'].nunique()

16

In [280]:
# These 16 buildings do not have a newer building area recorded around the same time as the renovation listed
# by the SSSP. The code years also do not show a renovation bringing the buildings up to a more recent code.
df_sssp_phase2_upgrade_missing

,building_id,District,Site,Building,upgrade_recorded,Year Built (SSSP),Year of Structural Upgrade (SSSP),Building_YearBuilt (ICOS),BuildingArea_YearBuilt (ICOS),BuildingArea_CodeYear (ICOS),BuildingArea_HasRetrofit (ICOS)
0,0,Aberdeen,A.J. West Elementary School,1952 Building,no,1952,1994,1952,1952,<1973,No
1,1,Aberdeen,A.J. West Elementary School,Annex Building,no,1966,1994,1966,1966,<1973,No
6,4,Bellingham,Fairhaven Middle School,Main Building,no,1937,1994,1937,1937,<1973,No
14,8,Chimacum,Chimacum Middle School,Middle School Bldg 100 B,no,1959,1999,1959,1959,<1973,No
23,12,Highline,Southern Heights Elementary School,Building A,no,1955,1987,1955,1955,<1973,No
24,13,Highline,Southern Heights Elementary School,Building B,no,1956,1987,1956,1956,<1973,No
25,14,Marysville,Cascade Elementary School,Unit A,no,1955,1972,1955,1955,<1973,No
26,15,Marysville,Cascade Elementary School,Unit C,no,1956,1972,1956,1956,<1973,No
27,16,Marysville,Cascade Elementary School,Unit D,no,1956,1972,1956,1956,<1973,No
28,17,Northshore,Crystal Springs Elementary School,Building 1 - Admin,no,1957,2010,1957,1990,Unknown,No


In [ ]:
# Out of 24 buildings in the SSSP phase 1 that are marked as having a prior seismic upgrade with a renovation year
# listed, 15 do not have a building area year around the renovation year, or a newer code year to show that the
# renovation recorded in the SSSP was incorporated into ICOS.

In [85]:
# Out of 34 buildings in the SSSP phase 2 that are marked as having a prior seismic upgrade with a renovation year
# listed, 16 do not have a building area year around the renovation year, or a newer code year to show that the
# renovation recorded in the SSSP was incorporated into ICOS.

In [ ]:
# 31 out of 58 buildings identified in both phases of the SSSP as having prior seismic upgrades
# do not show newer building years or newer code years indicating that a renovation was incorporated in in ICOS. 
# Only 5 out of the 58 buildings show a retrofit in ICOS.